In [ ]:
# Parameters
hashtag = "minecraft"


In [ ]:
import tweepy
import os
import json
from IPython.display import JSON
import logging
import parser
import traceback
import sys
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table


In [ ]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

def get_keywords():
    Base = automap_base()

    # engine, suppose it has two tables 'user' and 'address' set up
    engine = create_engine(os.environ['SHARED_DB_URI'])

    # reflect the tables
    Base.prepare(engine, reflect=True)

    # mapped classes are now created with names by default
    # matching that of the table name.

    raws = Base.classes.raws
    keywords = Base.classes.keywords

    session = Session(engine)

    keywords = session.query(keywords).all()
    
    return keyword

    for keyword in keywords:
        recordObject = {'id': keyword.id,
                        'keyword_name': keyword.keyword_name,
                        'brand_id': keyword.brand_id}
        print(recordObject)

Oauth handling (probably should do a class later). Shouldalso use an generator for secret and key but as twitter doesn't expire it it will do for the moment. 
In a production version I would probably generate an app by account so I can track each account usage. 

Oauth 2 could also psossibly be a solution as I only need public infos. 

In [ ]:
consumer_key = os.environ['TWITTER_API_KEY']
consumer_secret = os.environ['TWITTER_API_SECRET']
key = os.environ['TWITTER_ACCESS_TOKEN']
secret = os.environ['TWITTER_ACCESS_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(key, secret)
api = tweepy.API(auth)

# using tweepy 
The main objective is to gather infos from twitter. We're gonna use the stream endpoint because using the search doesn't return all the tweets.

Probably end up bumping into api limitations pretty fast but it will be ok for the demo.

The process will be the following : 

- Get latest tweet 
- Store Raw response (probably in an S3/minio bucket. will probably do it in second time but it should be a good idea in case the processing crashes
- get relevant datas
- run additional infos gathering and transform 
      - detect mood ? 
      - lang detection ? 
      - ...
- store in the sql base (shared) 

Airflow process : 
- Each hour detect new keywords. 
- For each keyord create a notebook (aka a thread) 
- Check if all notebook Running. 
- can mark a keyword as inactive in Flask then  don't check 


In [ ]:
#set up our main class using tweepy.StreamListener
class TwitterListener(tweepy.StreamListener):
    
    def __init__(self):
        self.database = os.environ['SHARED_DB_URI']

    def on_error(self, status_code):
        if status_code == 420:
            # returning False in on_data disconnects the stream
            return False

    def on_status(self, status):
        print(status.text)
        return True

    def on_data(self, data):
        """
        Automatic detection of the kind of data collected from Twitter
        This method reads in tweet data as JSON and extracts the data we want.
        """
        try:
            # parse as json
            raw_data = json.loads(data)

            #insert data just collected into MySQL my_database
            self.populate_table(raw_data)

        except Exception as e:
            logging.error(traceback.format_exc())
            # Logs the error appropriately. 
            

# todo : create the tables on init
    def populate_table(
        self, raw_data
    ):
        """Populate a given table witht he Twitter collected data

        Args:
            raw_data (json) : storing raw data for further usage
        """
        engine = create_engine(self.database)

        # Create connection
        conn = engine.connect()
        meta = MetaData()
        
        #get table
        raw_tweet = Table('raw_tweet', meta, autoload=True, autoload_with=engine)
        
        # Begin transaction
        trans = conn.begin()
        
        ins = raw_tweet.insert().values(rawtweet=raw_data)

        #actual content of request
        conn.execute(ins)

        try:
            trans.commit()

        except mysql.Error as e:
            print(e)
            trans.rollback()

        # Close connection
        conn.close()
        print(f"Tweet colleted")

        return

In [ ]:
tweepy_listener = TwitterListener()
tweepy_stream = tweepy.Stream(auth = api.auth, listener=tweepy_listener)
results = tweepy_stream.filter(track=['Minecraft'])

In [ ]:
print(results)